### Задание

В рамках этого итогового задания мы будем прогнозировать сердечную недостаточность.

Плана по выполнению задания не будет. 
Но есть несколько требований: 

- оберните весь конвейер преобразований в Pipeline

- подберите оптимальный вариант прогнозной модели с помощью GridSearchCV

- примените обученный на тренировочных данных конвейер к тестовым данным, никак не предобрабатывая их, а лишь загрузив из файла и отделив целевой признак от остальных

- получите на тестовой части качество не ниже 0.87 по метрике ROCAUC

Пояснение: если пропуски в новых данных и будут, то только в тех колонках, где они есть в тренировочной части.

In [522]:
import pandas as pd
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier

In [523]:
df_train = pd.read_csv('heart_adapt_train.csv')
df_train.head(10)

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,74.0,M,NAP,138.0,NaN,0,Normal,116,N,0.2,Up,0
1,58.0,M,NAP,132.0,224.0,0,LVH,173,N,3.2,Up,1
2,44.0,M,ATA,150.0,288.0,0,Normal,150,Y,3.0,Flat,1
3,50.0,M,ASY,144.0,349.0,0,LVH,120,Y,1.0,Up,1
4,NaN,M,ASY,145.0,248.0,0,Normal,96,Y,2.0,Flat,1
5,51.0,M,NAP,135.0,160.0,0,Normal,150,N,2.0,Flat,1
6,53.0,M,ASY,154.0,NaN,1,ST,140,Y,1.5,Flat,1
7,38.0,M,NAP,138.0,175.0,0,Normal,173,N,0.0,Up,0
8,56.0,M,NAP,125.0,NaN,1,Normal,98,N,-2.0,Flat,1
9,61.0,M,ASY,190.0,287.0,1,LVH,150,Y,2.0,Down,1


In [524]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 589 entries, 0 to 588
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             533 non-null    float64
 1   Sex             589 non-null    object 
 2   ChestPainType   589 non-null    object 
 3   RestingBP       588 non-null    float64
 4   Cholesterol     462 non-null    float64
 5   FastingBS       589 non-null    int64  
 6   RestingECG      589 non-null    object 
 7   MaxHR           589 non-null    int64  
 8   ExerciseAngina  589 non-null    object 
 9   Oldpeak         589 non-null    float64
 10  ST_Slope        589 non-null    object 
 11  HeartDisease    589 non-null    int64  
dtypes: float64(4), int64(3), object(5)
memory usage: 55.3+ KB


Пропуски присутствуют в числовых колонках 'Age', 'RestingBP' и 'Cholesterol'

In [525]:
df_train.describe()

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease
count,533.000000,588.000000,462.000000,589.000000,589.000000,589.000000,589.000000
mean,54.195122,133.358844,245.632035,0.258065,134.893039,0.937521,0.646859
std,9.532661,18.851852,58.599184,0.437942,24.942596,1.071318,0.478352
min,28.000000,80.000000,85.000000,0.000000,63.000000,-2.600000,0.000000
25%,48.000000,120.000000,209.000000,0.000000,117.000000,0.000000,0.000000
50%,55.000000,130.000000,240.000000,0.000000,135.000000,0.800000,1.000000
75%,61.000000,144.000000,279.750000,1.000000,154.000000,1.600000,1.000000
max,77.000000,200.000000,603.000000,1.000000,195.000000,5.000000,1.000000


Пропуски в числовых данных можно будет заменить на средние значения

In [526]:
# отделяем признаки от целевой переменной
features_train = df_train.drop(['HeartDisease'], axis=1)
target_train =df_train['HeartDisease']

In [527]:
target_train.value_counts()

HeartDisease
1    381
0    208
Name: count, dtype: int64

Целевая переменная не сбалансирована

In [528]:
# класс для приведения категориальных переменных к числовому типу
class CategoriesToFlags(TransformerMixin, BaseEstimator):

    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return pd.get_dummies(X, drop_first=True)

In [529]:
# загружаем тестовые данные
df_test = pd.read_csv('heart_adapt_test.csv')
df_test.head(10)

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,44.0,M,NAP,130.0,233.0,0,Normal,179,Y,0.4,Up,0
1,63.0,M,ASY,130.0,308.0,0,Normal,138,Y,2.0,Flat,1
2,35.0,F,TA,120.0,160.0,0,ST,185,N,0.0,Up,0
3,69.0,M,NAP,140.0,NaN,1,ST,118,N,2.5,Down,1
4,NaN,M,TA,142.0,200.0,1,ST,100,N,1.5,Down,1
5,57.0,M,ASY,140.0,214.0,0,ST,144,Y,2.0,Flat,1
6,NaN,M,ASY,130.0,283.0,1,LVH,103,Y,1.6,Down,1
7,42.0,M,ASY,145.0,NaN,0,Normal,99,Y,0.0,Flat,1
8,56.0,M,NAP,130.0,167.0,0,Normal,114,N,0.0,Up,0
9,46.0,M,ASY,110.0,202.0,0,Normal,150,Y,0.0,Flat,1


In [530]:
# отделяем признаки от целевой переменной
features_test = df_test.drop(['HeartDisease'], axis=1)
target_test =df_test['HeartDisease']

In [531]:
# создаем Pipeline
pipe = Pipeline(steps=[
    ('ohe_types', CategoriesToFlags()), # приведение категориальных признаков к числовому типу
    ('imputer', SimpleImputer(strategy='mean')),  # заполнение пропусков
    ('scaler', StandardScaler()),  # нормализация признаков
    ('classify', DecisionTreeClassifier(class_weight='balanced', random_state=0))  # создание модели
])


In [532]:
pipe.fit(X=features_train, y=target_train)
print('Качество модели на обучающей выборке:', {roc_auc_score(target_train, pipe.predict_proba(features_train)[:, 1])})

print('Качество модели на тестовой выборке:', {roc_auc_score(target_test, pipe.predict_proba(features_test)[:, 1])})

Качество модели на обучающей выборке: {np.float64(1.0)}
Качество модели на тестовой выборке: {np.float64(0.8305961754780653)}


In [533]:
params = [
    {'classify': [LogisticRegression(class_weight='balanced', random_state=0)]}, 
    {'classify': [RandomForestClassifier(class_weight='balanced', random_state=42)], 'classify__n_estimators': [50, 100],
                                                                                    'classify__max_depth': [None, 10, 20],
                                                                                    'classify__min_samples_split': [2, 5]}
]
grid_search = GridSearchCV(pipe, param_grid=params, cv=5, scoring='roc_auc')
grid_search.fit(X=features_train, y=target_train)

print('Качество модели на тестовой выборке c лучшей моделью:', 
    {roc_auc_score(target_test, grid_search.predict_proba(features_test)[:, 1])})
print('Параметры лучшей модели:')
for k in grid_search.best_params_.keys():
    print(f'   --- {k} : {grid_search.best_params_[k]}')

Качество модели на тестовой выборке c лучшей моделью: {np.float64(0.9109111361079866)}
Параметры лучшей модели:
   --- classify : RandomForestClassifier(class_weight='balanced', random_state=42)
   --- classify__max_depth : None
   --- classify__min_samples_split : 2
   --- classify__n_estimators : 50


### Вывод:

При создании модели прогнозирования сердечной недостаточности удалось достичь качество на тестовой выборке 0.9109111361079866.
Лучшей моделью оказалась RandomForestClassifier с со следующими гиперпараметрами:
- max_depth = None
- min_samples_split = 2
- n_estimators = 50